In this experiment you will generate traffic using OSNT and capture it with DAG.  
###### Setup:  
-- `bash /root/<crsid>/L50/setup/load_osnt_bitfile.sh <crsid>` (Machine B)    
-- (Reboot and reload bitfile if necessary.)  
-- `dagload` (Machine A)   
-- `ssh-add` (Machine A)    
-- Connect nf0 (B) to dag0 (A).     
-- Connect nf1 (B) to nf2 (B).     
-- `bash /root/<crsid>/L50/setup/mkdir3.sh <crsid>`. (Machine A)  
<img src="alt_setup3.1.jpg">

OSNT takes a pcap file along with other parameters, to determine how to replay traffic. pcap files are packet capture files, which are loaded into our traffic generation tool to be replayed. You can view the details by opening them in Wireshark. There are prepared pcap files in L50/pcap_files to use for this experiment.

###### Usage:    
For nf0, `python osnt-tool-cmd.py -ifp0 <pcap file> -rpn0 <replay number> [Optional: -ipg0 <gap (ns)>] -run `  
NB. ipg in OSNT is the gap in nanoseconds between the start of one packet and the start of the next. If the supplied value is too small it is ignored; the default value is 0.  For 10 Gbit Ethernet, 10 bits travel in 1 ns.  
NB. This command exits immediately, which may be before the packets have finished replaying. So you must sleep a certain amount of time before killing the capturing process.  
  
Eg. Replay blah.pcap 20 times on nf1, with gap 100000 ns.  
`python osnt-tool-cmd.py -ifp1 blah.pcap -rpn1 20 -ipg1 100000 -run`  

In [ ]:
crsid=''
%run /root/$crsid/L50/Jupyter/useful/useful.py
%run /root/$crsid/L50/Jupyter/useful/useful3.py
%matplotlib inline
machB_ip = '' # REPLACE THIS

The function `send` will be used throughout this Notebook:  
Generate traffic from the NetFPGA card (Machine B) using OSNT. Traffic to the DAG card (Machine A) is captured to a .erf file. One summary line for each packet received is written to a .txt file. Check that these are correctly saved to L50Lab3/3.1.   


In [ ]:
def send(exp,size,num,ipg=0):
    # size = packet size in bytes
    # num = no of packets to send
    # ipg = ipg in OSNT
    
    cmd = shlex.split('dagsnap -d0 -o /root/'+crsid+'/L50Lab3/3.1/'+exp+'.erf')
    p = Popen(cmd) # start capture
    sleep(2)
    play_cmd = ('cd /root/OSNT-SUME-live/projects/osnt/sw/host/app/cli && python osnt-tool-cmd.py '
                '-ifp0 /root/'+crsid+'/L50/pcap_files/'+str(size)+'.cap -rpn0 '+str(num)+' -ipg0 '+str(ipg)+' -run')
    ssh = ssh_connect(machB_ip)
    ssh_cmd(play_cmd,ssh)
    ssh.close()
    sleep(5)
    p.kill() # end capture
    sleep(2)
    local_cmd('tshark -r /root/'+crsid+'/L50Lab3/3.1/'+exp+'.erf > /root/'+crsid+'/L50Lab3/3.1/'+exp+'.txt') # write summary lines

## Experiment a

Send 1000 packets, size 512B. See that all the packets arrive.

In [ ]:
%%capture
send('exp1a',512,1000)

In [ ]:
num_pkts = sum(1 for line in open('/root/'+crsid+'/L50Lab3/3.1/exp1a.txt') if line)
print "Packets received: " + str(num_pkts)

## Experiment b

Send 100000 packets, size 512B. See that all the packets arrive. What is the captured packet rate?

In [ ]:
%%capture
send('exp1b',512,100000)

In [ ]:
num_pkts = sum(1 for line in open('/root/'+crsid+'/L50Lab3/3.1/exp1b.txt') if line)
print "Packets received: " + str(num_pkts)

deltas = getdeltas('3.1/exp1b',crsid,100000)
pps = int(floor(1.0 / np.average(deltas) * 1000000))
print "Packets per second: " + str(pps)

## Experiment c

Send 100000 packets, size 1024B / 512B / 64B. What is the maximum packet rate supported by the tool? What is the bandwidth? What is the effect of line coding (64b/66b)?

In [ ]:
def rate(exp,size,num):
    deltas = getdeltas(exp,crsid,num)
    rt = (size+20+4)*8/ np.average(deltas)
    print "BW: " + str(rt) + " Mbits/sec" 
    pps = int(floor(1.0 / np.average(deltas) * 1000000))
    print "Packets per second: " + str(pps)

### 1024B

In [ ]:
%%capture
send('exp1c1',1024,100000)

In [ ]:
rate('3.1/exp1c1',1024,100000)

### 512B

In [ ]:
%%capture
send('exp1c2',512,100000)

In [ ]:
rate('3.1/exp1c2',512,100000)

### 64B

In [ ]:
%%capture
send('exp1c3',64,100000)

In [ ]:
rate('3.1/exp1c3',64,100000)

## Experiment d

Send 100000 packets, size 512B, 100 Mbits/sec. What is the measured traffic throughput?

In [ ]:
ipg = (10000/100)*(512+4+20)*8/10
print "ipg: " + str(ipg)
send('exp1d',512,100000,ipg)

In [ ]:
rate('3.1/exp1d', 512,100000)

## Experiment e

What is the cumulative distribution function (CDF) of inter-arrival time of experiment d?

In [ ]:
deltas = getdeltas('3.1/exp1d',crsid,100000)
values, base = np.histogram(deltas, bins=100)
cumulative = np.cumsum(values/99999.0)
plt.plot(base[:-1], cumulative)
plt.ylabel("Cumulative probability")
plt.xlabel("Inter-arrival time (us)")
plt.show()

## Experiment f

Send 100000 packets, size 512B, nf0 to DAG, nf1 to nf2, 100 Mbits/sec. What is the distribution of inter-arrival time?

In [ ]:
%%capture

cmd=shlex.split('dagsnap -d0 -o /root/'+crsid+'/L50Lab3/3.1/exp1f.erf')
p = Popen(cmd) #start capture
sleep(2)
ipg = (10000/100)*(512+4+20)*8/10
play_cmd = ("cd /root/OSNT-SUME-live/projects/osnt/sw/host/app/cli && python osnt-tool-cmd.py "
            "-ifp0 /root/"+crsid+"/L50/pcap_files/512.cap -rpn0 100000 -ipg0 "+str(ipg)+
            " -ifp1 /root/"+crsid+"/L50/pcap_files/512.cap -rpn1 100000 -ipg1 " +str(ipg)+" -run")
ssh = ssh_connect(machB_ip)
ssh_cmd(play_cmd,ssh) # generate traffic
ssh.close()
sleep(5)
p.kill() # end capture
sleep(2)

Plot the CDF of inter-arrival time.

In [ ]:
deltas2 = getdeltas('3.1/exp1f',crsid,100000)
values, base = np.histogram(deltas2, bins=100)
cumulative = np.cumsum(values/99999.0)
plt.plot(base[:-1], cumulative)
plt.ylabel("Cumulative probability")
plt.xlabel("Inter-arrival time (us)")
plt.show()